In [1]:
import pandas as pd

In [2]:
%run sqlite_conn.ipynb

In [3]:
# Create a db connection
database = r"C:\Users\hippo\wd\work_stuff\esg\easy_side_games.db"
conn = create_connection(database)

In [4]:
"""
Putting this at the top for visibility.
If these are metrics the business always wants it would make more sense to persist this data as a table in a datamart.
"""

bonus_query = """
    SELECT 
        PROB_A.DATE
        ,PROB_A.DAU
        ,PROB_B.INSTALLS
        ,PROB_C.REV
        ,PROB_D.CONV
        ,PROB_E.ARPAU
        ,PROB_F.D1
    FROM (
            SELECT
                PLAY_DATE AS DATE
                ,COUNT(DISTINCT USER_ID) as DAU
            FROM SESSIONS_BEGIN
            GROUP BY PLAY_DATE) AS PROB_A
    LEFT JOIN (
            SELECT
                INSTALL_DATE as DATE
                ,COUNT(DISTINCT USER_ID) as INSTALLS
            FROM INSTALL
            GROUP BY INSTALL_DATE) as PROB_B ON PROB_A.DATE = PROB_B.DATE
    LEFT JOIN (
            SELECT
                PURCHASE_DATE AS DATE
                ,SUM(amount) as REV
            FROM PURCHASE
            GROUP BY PURCHASE_DATE) as PROB_C ON PROB_A.DATE = PROB_C.DATE
    LEFT JOIN (
            SELECT
                PLAY_DATE AS DATE
                ,COUNT(DISTINCT PUR.USER_ID)*1.0/COUNT(DISTINCT SB.USER_ID) as CONV
            FROM SESSIONS_BEGIN as SB
            LEFT JOIN PURCHASE as PUR ON SB.PLAY_DATE = PUR.PURCHASE_DATE and SB.USER_ID = PUR.USER_ID
            GROUP BY PLAY_DATE) as PROB_D ON PROB_A.DATE = PROB_D.DATE
    LEFT JOIN (
            SELECT
                PLAY_DATE AS DATE
                ,SUM(PUR.AMOUNT)/COUNT(DISTINCT SB.USER_ID) as ARPAU
            FROM SESSIONS_BEGIN as SB
            LEFT JOIN PURCHASE as PUR ON SB.PLAY_DATE = PUR.PURCHASE_DATE and SB.USER_ID = PUR.USER_ID
            GROUP BY PLAY_DATE) as PROB_E ON PROB_A.DATE = PROB_E.DATE
    LEFT JOIN (
            SELECT
                INS.INSTALL_DATE AS DATE
                ,COUNT(DISTINCT SB.USER_ID)*1.0/COUNT(DISTINCT INS.USER_ID) AS D1
            FROM INSTALL AS INS
            LEFT JOIN SESSIONS_BEGIN as SB ON INS.USER_ID = SB.USER_ID and DATE(SB.PLAY_DATE) = DATE(INS.INSTALL_DATE, '+1 day')
            GROUP BY INS.INSTALL_DATE) as PROB_F ON PROB_A.DATE = PROB_F.DATE
"""

bonus_df = pd.read_sql_query(bonus_query, conn)
print(bonus_df)

          DATE  DAU  INSTALLS       REV      CONV     ARPAU        D1
0   2018-07-01  178       168   8995.78  0.044944  1.962191  0.035714
1   2018-07-02  152       171   9159.33  0.059211  3.068421  0.035088
2   2018-07-03  161       165   9820.76  0.049689  2.542857  0.030303
3   2018-07-04  163       164   7541.63  0.036810  1.963374  0.030488
4   2018-07-05  170       156   8851.94  0.047059  3.547353  0.044872
5   2018-07-06  188       195   7358.08  0.026596  1.214096  0.025641
6   2018-07-07  199       186   8986.02  0.020101  0.707789  0.026882
7   2018-07-08  173       173   8430.47  0.040462  2.304220  0.023121
8   2018-07-09  163       170   8562.93  0.024540  1.933988  0.047059
9   2018-07-10  153       187   9817.71  0.039216  1.368954  0.042781
10  2018-07-11  120       205   8967.06  0.033333  1.521167  0.024390
11  2018-07-12  183       179   9854.30  0.038251  2.658087  0.033520
12  2018-07-13  151       160   8789.94  0.033113  1.682980  0.018750
13  2018-07-14  212 

In [5]:
problem_a_query = """
    SELECT
        PLAY_DATE AS DATE
        ,COUNT(DISTINCT USER_ID) as DAU
    FROM SESSIONS_BEGIN
    GROUP BY PLAY_DATE
"""

problem_a_df = pd.read_sql_query(problem_a_query, conn)
print(problem_a_df)

          DATE  DAU
0   2018-07-01  178
1   2018-07-02  152
2   2018-07-03  161
3   2018-07-04  163
4   2018-07-05  170
5   2018-07-06  188
6   2018-07-07  199
7   2018-07-08  173
8   2018-07-09  163
9   2018-07-10  153
10  2018-07-11  120
11  2018-07-12  183
12  2018-07-13  151
13  2018-07-14  212
14  2018-07-15  146
15  2018-07-16  186
16  2018-07-17  161
17  2018-07-18  179
18  2018-07-19  168
19  2018-07-20  202
20  2018-07-21  170
21  2018-07-22  205
22  2018-07-23  178
23  2018-07-24  174
24  2018-07-25  187
25  2018-07-26  153
26  2018-07-27  171
27  2018-07-28  171
28  2018-07-29  183


In [6]:
problem_b_query = """
    SELECT
        INSTALL_DATE as DATE
        ,COUNT(DISTINCT USER_ID) as INSTALLS
    FROM INSTALL
    GROUP BY INSTALL_DATE
"""

problem_b_df = pd.read_sql_query(problem_b_query, conn)
print(problem_b_df)

          DATE  INSTALLS
0   2018-07-01       168
1   2018-07-02       171
2   2018-07-03       165
3   2018-07-04       164
4   2018-07-05       156
5   2018-07-06       195
6   2018-07-07       186
7   2018-07-08       173
8   2018-07-09       170
9   2018-07-10       187
10  2018-07-11       205
11  2018-07-12       179
12  2018-07-13       160
13  2018-07-14       167
14  2018-07-15       171
15  2018-07-16       179
16  2018-07-17       176
17  2018-07-18       164
18  2018-07-19       178
19  2018-07-20       140
20  2018-07-21       152
21  2018-07-22       176
22  2018-07-23       182
23  2018-07-24       168
24  2018-07-25       168
25  2018-07-26       180
26  2018-07-27       172
27  2018-07-28       172
28  2018-07-29       176


In [7]:
problem_c_query = """
    SELECT
        PURCHASE_DATE AS DATE
        ,SUM(amount) as REV
    FROM PURCHASE
    GROUP BY PURCHASE_DATE
"""

problem_c_df = pd.read_sql_query(problem_c_query, conn)
print(problem_c_df)

          DATE       REV
0   2018-07-01   8995.78
1   2018-07-02   9159.33
2   2018-07-03   9820.76
3   2018-07-04   7541.63
4   2018-07-05   8851.94
5   2018-07-06   7358.08
6   2018-07-07   8986.02
7   2018-07-08   8430.47
8   2018-07-09   8562.93
9   2018-07-10   9817.71
10  2018-07-11   8967.06
11  2018-07-12   9854.30
12  2018-07-13   8789.94
13  2018-07-14   8726.58
14  2018-07-15   8974.72
15  2018-07-16   9462.66
16  2018-07-17   8658.67
17  2018-07-18   8093.38
18  2018-07-19  10098.56
19  2018-07-20   9191.88
20  2018-07-21   8585.44
21  2018-07-22   8111.36
22  2018-07-23   8653.34
23  2018-07-24   7642.76
24  2018-07-25   8266.76
25  2018-07-26   8836.78
26  2018-07-27   8344.46
27  2018-07-28   9214.27
28  2018-07-29   8656.72


In [8]:
problem_d_query = """
    SELECT
        PLAY_DATE AS DATE
        ,COUNT(DISTINCT PUR.USER_ID)*1.0/COUNT(DISTINCT SB.USER_ID) as CONV
    FROM SESSIONS_BEGIN as SB
    LEFT JOIN PURCHASE as PUR ON SB.PLAY_DATE = PUR.PURCHASE_DATE and SB.USER_ID = PUR.USER_ID
    GROUP BY PLAY_DATE
    
"""

problem_d_df = pd.read_sql_query(problem_d_query, conn)
print(problem_d_df)

          DATE      CONV
0   2018-07-01  0.044944
1   2018-07-02  0.059211
2   2018-07-03  0.049689
3   2018-07-04  0.036810
4   2018-07-05  0.047059
5   2018-07-06  0.026596
6   2018-07-07  0.020101
7   2018-07-08  0.040462
8   2018-07-09  0.024540
9   2018-07-10  0.039216
10  2018-07-11  0.033333
11  2018-07-12  0.038251
12  2018-07-13  0.033113
13  2018-07-14  0.037736
14  2018-07-15  0.041096
15  2018-07-16  0.032258
16  2018-07-17  0.031056
17  2018-07-18  0.022346
18  2018-07-19  0.053571
19  2018-07-20  0.039604
20  2018-07-21  0.052941
21  2018-07-22  0.024390
22  2018-07-23  0.050562
23  2018-07-24  0.022989
24  2018-07-25  0.026738
25  2018-07-26  0.026144
26  2018-07-27  0.029240
27  2018-07-28  0.035088
28  2018-07-29  0.027322


In [9]:
problem_e_query = """
    SELECT
        PLAY_DATE AS DATE
        ,SUM(PUR.AMOUNT)/COUNT(DISTINCT SB.USER_ID) as ARPAU
    FROM SESSIONS_BEGIN as SB
    LEFT JOIN PURCHASE as PUR ON SB.PLAY_DATE = PUR.PURCHASE_DATE and SB.USER_ID = PUR.USER_ID
    GROUP BY PLAY_DATE
    
"""

problem_e_df = pd.read_sql_query(problem_e_query, conn)
print(problem_e_df)

          DATE     ARPAU
0   2018-07-01  1.962191
1   2018-07-02  3.068421
2   2018-07-03  2.542857
3   2018-07-04  1.963374
4   2018-07-05  3.547353
5   2018-07-06  1.214096
6   2018-07-07  0.707789
7   2018-07-08  2.304220
8   2018-07-09  1.933988
9   2018-07-10  1.368954
10  2018-07-11  1.521167
11  2018-07-12  2.658087
12  2018-07-13  1.682980
13  2018-07-14  1.176792
14  2018-07-15  1.626575
15  2018-07-16  1.458656
16  2018-07-17  1.457205
17  2018-07-18  1.203799
18  2018-07-19  1.868393
19  2018-07-20  2.108317
20  2018-07-21  3.177882
21  2018-07-22  1.053024
22  2018-07-23  3.337921
23  2018-07-24  0.847471
24  2018-07-25  1.503369
25  2018-07-26  1.404575
26  2018-07-27  1.497836
27  2018-07-28  1.681637
28  2018-07-29  0.697596


In [10]:
"""
Note here the dates don't make sense across tables so this metric looks low but the query is correct.
Also note the last value for d1 will always be zero because I only generated 30 days of dumy data.
"""

problem_f_query = """
    SELECT
        INS.INSTALL_DATE AS DATE
        ,COUNT(DISTINCT SB.USER_ID)*1.0/COUNT(DISTINCT INS.USER_ID) AS D1
    FROM INSTALL AS INS
    LEFT JOIN SESSIONS_BEGIN as SB ON INS.USER_ID = SB.USER_ID and DATE(SB.PLAY_DATE) = DATE(INS.INSTALL_DATE, '+1 day')
    GROUP BY INS.INSTALL_DATE
    
"""

problem_f_df = pd.read_sql_query(problem_f_query, conn)
print(problem_f_df)

          DATE        D1
0   2018-07-01  0.035714
1   2018-07-02  0.035088
2   2018-07-03  0.030303
3   2018-07-04  0.030488
4   2018-07-05  0.044872
5   2018-07-06  0.025641
6   2018-07-07  0.026882
7   2018-07-08  0.023121
8   2018-07-09  0.047059
9   2018-07-10  0.042781
10  2018-07-11  0.024390
11  2018-07-12  0.033520
12  2018-07-13  0.018750
13  2018-07-14  0.023952
14  2018-07-15  0.046784
15  2018-07-16  0.044693
16  2018-07-17  0.017045
17  2018-07-18  0.060976
18  2018-07-19  0.028090
19  2018-07-20  0.028571
20  2018-07-21  0.032895
21  2018-07-22  0.028409
22  2018-07-23  0.054945
23  2018-07-24  0.029762
24  2018-07-25  0.023810
25  2018-07-26  0.038889
26  2018-07-27  0.046512
27  2018-07-28  0.075581
28  2018-07-29  0.000000
